## Preuzimanje podataka sa portala Sunny portal

In [ ]:
#%pip install requests
#%pip install bs4
#%%pip install lxml

In [3]:
import pandas as pd
import datetime as dt
import numpy as np
import requests
from bs4 import BeautifulSoup

Za preuzimanje podataka sa web stranice, prvo je potrebno odrediti godine za koje će se preuzeti, te formirati rang datuma od početka pa do kraja godine u formatu M/D/YYYY jer se takav format pojavljuje u linkovima na web-stranici. Za ovo služi funkcija **define_year**.

In [4]:
def define_year(year,number_of_days1):
    start_date = dt.date(year, 1 , 1)
    number_of_days = number_of_days1
    date_list = [((start_date + dt.timedelta(days = day)).strftime("%#m/%#d/%Y")) for day in range(number_of_days)]
    #date_list2=[((start_date + dt.timedelta(days = day)).strftime("%Y-%m-%d")) for day in range(number_of_days)]
    return date_list

Podaci će se ekstraktovati na način da se link web-stranice mijenja u zavisnosti od datuma koji je postavljen u date_list_1, te će se u HTML fajlu (page source) naći tabela sa podacima proizvodnje električne energije za taj datum. Za ovo je potrebno imati link koji odgovara svakoj solarnoj elektrani pojedinačno. Za jednu solarnu elektranu su svi dijelovi linka isti izuzev datuma koji se mijenja. Prema tome, kada se napravi petlja kojom se mijenja datum i ubacuje u ostatak linka, na HTML stranici se pronalazi odgovarajuća tabela sa podacima i spašava za dalje procesiranje. Ovaj dio se odrađuje putem funkcije **extract_data** koja kao ulazne podatke prima datume u jednoj godini, te dva dijela linka izmđeu kojih će se taj datum ubaciti.

In [5]:
def extract_data(date_list_1,link1,link2):
    list_csvs=[None]*len(date_list_1)

    for k,value in enumerate(date_list_1):
        #link=link1+value+link2
        link=link1+value+link2
        
        res = requests.get(link)

#print(res.text)
#print(res.status_code)

        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
    #page_title = soup.title.text
    #print(page_title)

        table1 = soup.find('table', id='ctl00_ContentPlaceHolder1_UserControlChartValues1_Table1')
    	#table1

 
        csvContent = ''
        for j in table1.find_all('tr'):
            row_data = j.find_all('td')

            for cell in row_data:
                csvContent = csvContent + cell.text + ','
            csvContent= csvContent[:-1]
            csvContent = csvContent + '\n'
    
        #print(csvContent,k)
        list_csvs[k]=csvContent
    return list_csvs
    

Prikupljeni podaci se nalaze u tekstualnom obliku, te ih je potrebno pretvoriti u tabelarni prikaz kako bi se mogli spasiti u .xls formatu. list_csvs je lista tekstualnih podataka za jednu čitavu godinu (ima 365, odnosno 366 elemenata za jednu godinu). Ti elementi se najprije pretvaraju u tabele putem funkcije **write_to_dataframe**, a zatim se te tabele spajaju u jednu putem funkcije **concat_and_save**.

In [6]:
def write_to_dataframe(list_csvs):
    dfs=[None]*len(list_csvs)
    for k,value in enumerate(list_csvs):
        with open('output.csv','w',newline='') as f:
            f.write(list_csvs[k])
        dfs[k]=pd.read_csv("output.csv")
    return dfs

In [7]:
def concat_and_save(dfs,name):
    df_tot=pd.concat(dfs,axis=0)
    df_tot.to_excel(name)
    return df_tot

In [8]:
#enelectronic
link_ene1="https://www.sunnyportal.com/Templates/PublicChartValues.aspx?ID=6e2898bd-52c5-455a-81f8-1c5c492aa03f&endTime="
link_ene2="%2011:59:59%20PM&splang=en-US&plantTimezoneBias=60&name="
#krenica i hodovo se nalaze u istom fajlu
link_kr_hod1="https://www.sunnyportal.com/Templates/PublicChartValues.aspx?ID=0ada0c8e-6bcd-483b-9798-1bc3ff24b6a7&endTime=" 
link_kr_hod2="%2011:59:59%20PM&splang=en-US&plantTimezoneBias=60&name="

#definiranje godine i opsega datuma od početka do kraja
date_list_ene2019=define_year(2019,365)
date_list_ene2020=define_year(2020,366)
date_list_ene2021=define_year(2021,365)


date_list_kr_hod2019=define_year(2019,365)
date_list_kr_hod2020=define_year(2020,366)
date_list_kr_hod2021=define_year(2021,365)

#izvlačenje podataka  - Enelectronic 2019
list_csvs_ene2019=extract_data(date_list_ene2019,link_ene1,link_ene2)


Način pohrane podataka za Enelectronic je takav da spašava podatke za 48 sati umjesto za 24, te će se radi izbjegavanja dupliciranja datuma preuzeti svaki drugi element liste list_csvs_ene i spasiti u **listu new_list_ene**. To znači da će podaci biti spašeni kao 1.1.2019, 2.1.2019.,3.1.2019 umjesto 1.1.2019,1.1.2019.,2.1.2019.,2.1.2019.  Za FNE Krenica1 i Hodovo1 podaci su spašeni u opsegu za 24 sata, te za njih ovaj korak nije potreban.

In [11]:

new_list_ene_2019=[]

for value in range(0,len(list_csvs_ene2019),2):
    new_list_ene_2019.append(list_csvs_ene2019[value])

In [12]:
#kreiranje tabelarnog prikaza i spašavanje u .xlsx formatu

df_ene_2019=write_to_dataframe(new_list_ene_2019)
df_ene_2019_tot=concat_and_save(df_ene_2019,"enelectronic_2019.xlsx")

In [14]:
#ponavljanje postupka za 2020. i 2021.godinu
list_csvs_ene2020=extract_data(date_list_ene2020,link_ene1,link_ene2)
list_csvs_ene2021=extract_data(date_list_ene2021,link_ene1,link_ene2)

In [21]:
#izvlačenje podataka  - krenica, hodovo za sve 3 godine
list_csvs_kr_hod_2019=extract_data(date_list_kr_hod2019,link_kr_hod1,link_kr_hod2)

In [25]:
list_csvs_kr_hod_2020=extract_data(date_list_kr_hod2020,link_kr_hod1,link_kr_hod2)

In [23]:
list_csvs_kr_hod_2021=extract_data(date_list_kr_hod2021,link_kr_hod1,link_kr_hod2)

In [19]:
#ponavljanje postupka za 2020.2021 godinu - Enelectronic - preuredit će se u funckiju

new_list_ene_2020=[]
new_list_ene_2021=[]
for value in range(0,len(list_csvs_ene2020),2):
    new_list_ene_2020.append(list_csvs_ene2020[value])
    
for value in range(0,len(list_csvs_ene2021),2):
    new_list_ene_2021.append(list_csvs_ene2021[value])

In [20]:
df_ene_2020=write_to_dataframe(new_list_ene_2020)
df_ene_2020_tot=concat_and_save(df_ene_2020,"enelectronic_2020.xlsx")

df_ene_2021=write_to_dataframe(new_list_ene_2021)
df_ene_2021_tot=concat_and_save(df_ene_2021,"enelectronic_2021.xlsx")



In [22]:
df_kr_hod_2019=write_to_dataframe(list_csvs_kr_hod_2019)
df_kr_hod_2019_tot=concat_and_save(df_kr_hod_2019,"kr_hod_2019.xlsx")

In [26]:
df_kr_hod_2020=write_to_dataframe(list_csvs_kr_hod_2020)
df_kr_hod_2020_tot=concat_and_save(df_kr_hod_2020,"kr_hod_2020.xlsx")

In [24]:
df_kr_hod_2021=write_to_dataframe(list_csvs_kr_hod_2021)
df_kr_hod_2021_tot=concat_and_save(df_kr_hod_2021,"kr_hod_2021.xlsx")

In [28]:


link_breg1="https://www.sunnyportal.com/Templates/PublicChartValues.aspx?ID=cd16e224-fd2d-43c6-9da5-2ad6719e68dd&endTime="
link_breg2="%2011:59:59%20PM&splang=en-US&plantTimezoneBias=60&name="
#elektrana počela sa radom u 2019. 22.8.2019

date_list_breg2020=define_year(2020,366)
date_list_breg2021=define_year(2021,365)


In [38]:
start_date = dt.date(2019, 8 , 22)
number_of_days = 133
date_list_breg2019 = [((start_date + dt.timedelta(days = day)).strftime("%#m/%#d/%Y")) for day in range(number_of_days)]

In [40]:
list_csvs_breg2019=extract_data(date_list_breg2019,link_breg1,link_breg2)

In [41]:
new_list_breg_2019=[]

for value in range(0,len(list_csvs_breg2019),2):
    new_list_breg_2019.append(list_csvs_breg2019[value])

In [42]:
df_breg_2019=write_to_dataframe(new_list_breg_2019)
df_breg_2019_tot=concat_and_save(df_breg_2019,"breg_2019.xlsx")

In [29]:
new_list_breg_2020=[]

for value in range(0,len(list_csvs_breg2020),2):
    new_list_breg_2020.append(list_csvs_breg2020[value])

In [ ]:
list_csvs_breg2020=extract_data(date_list_breg2020,link_breg1,link_breg2)

In [30]:
#kreiranje tabelarnog prikaza i spašavanje u .xlsx formatu

df_breg_2020=write_to_dataframe(new_list_breg_2020)
df_breg_2020_tot=concat_and_save(df_breg_2020,"breg_2020.xlsx")

In [31]:
list_csvs_breg2021=extract_data(date_list_breg2021,link_breg1,link_breg2)

In [32]:
new_list_breg_2021=[]

for value in range(0,len(list_csvs_breg2021),2):
    new_list_breg_2021.append(list_csvs_breg2021[value])
    #kreiranje tabelarnog prikaza i spašavanje u .xlsx formatu

df_breg_2021=write_to_dataframe(new_list_breg_2021)
df_breg_2021_tot=concat_and_save(df_breg_2021,"breg_2021.xlsx")